In [31]:
import pandas as pd  
import collections
import numpy as np
    
# making dataframe
df = pd.read_csv("sentiment140_with_location.csv", names=['target','id','date','flag','user','text', 'location'])  

df

,target,id,date,flag,user,text,location
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",China
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,Dominican Republic
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,Germany
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,Brazil
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",Argentina
...,...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,Japan
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,Spain
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Nigeria
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Argentina


In [32]:
# split dataset
shuffled = df.sample(frac = 1)
result = np.array_split(shuffled, 8)
result[0]

,target,id,date,flag,user,text,location
674717,0,2247873932,Fri Jun 19 19:45:24 PDT 2009,NO_QUERY,13hundred,55 -3 limp I limp man on left all in -I call h...,China
1235637,4,1992645999,Mon Jun 01 09:05:34 PDT 2009,NO_QUERY,CathrineWilloch,i think iÂ´m like .. melting . its so freakin ...,Ecuador
122624,0,1833746634,Mon May 18 00:18:54 PDT 2009,NO_QUERY,mcraddictal,I'm so sleepy. i have school tomorrow. I'm so...,Antarctica
207756,0,1973560194,Sat May 30 11:23:22 PDT 2009,NO_QUERY,ambermoffett,The worse realization is knowing thats it's no...,Australia
159876,0,1956941768,Thu May 28 23:05:12 PDT 2009,NO_QUERY,VeroHernandez,Snoopy just past by in the Harley I can't beli...,UK
...,...,...,...,...,...,...,...
925610,4,1755377755,Sun May 10 08:59:18 PDT 2009,NO_QUERY,kylefaticoni,"@seajayinla back off sister, justin timberlake...",Mexico
1526254,4,2177003242,Mon Jun 15 05:14:28 PDT 2009,NO_QUERY,rawrvee,"Today was tiiiight. A little tiring, but just...",Ethiopia
240698,0,1980937442,Sun May 31 08:02:17 PDT 2009,NO_QUERY,spygirl,Sitting down now. I just collapsed from Mega ...,Ethiopia
1098587,4,1970496190,Sat May 30 04:05:57 PDT 2009,NO_QUERY,bianxx_smile,i'd like to have a request a song from Demi `b...,India


In [5]:
# tweets_by_location = collections.defaultdict(list)
# sentiment_sum = 0
# for row in df.iterrows():
#     sentiment = row[1][0]
#     text = row[1][5]
#     location = row[1][6]
#     sentiment_sum += sentiment
#     tweets_by_location[location].append(text)
# # print(tweets_by_location)
# print("average sentiment: ", sentiment_sum/df.size)
    

average sentiment:  0.0


In [34]:
target_counts = collections.Counter(result[0]['target'])
location_counts = collections.Counter(result[0]['location'])
print("target counts: ", target_counts)
print("location_counts: ", location_counts)

target counts:  Counter({4: 100065, 0: 99935})
location_counts:  Counter({'UK': 6267, 'USA': 6203, 'Argentina': 6196, 'Ethiopia': 6179, 'Germany': 6177, 'Thailand': 6164, 'Spain': 6141, 'France': 6122, 'Kenya': 6118, 'Antarctica': 6115, 'Japan': 6093, 'Italy': 6092, 'Peru': 6089, 'Dominican Republic': 6076, 'New Zealand': 6076, 'Canada': 6072, 'Egypt': 6053, 'South Africa': 6053, 'Colombia': 6039, 'Ecuador': 6032, 'Nigeria': 6024, 'Indonesia': 6019, 'Mexico': 6018, 'Tanzania': 6008, 'Chile': 6008, 'Australia': 6007, 'Cuba': 5979, 'Russia': 5958, 'South Korea': 5949, 'China': 5940, 'Brazil': 5927, 'India': 5916, 'Jamaica': 5890})


In [46]:
def locationSentiment(data, location):
    """
    input: location is the input location of interest
    output: the average sentiment of the tweets for a location
    """
    # go through the dataframe filtered by location
    # get the average sentiment of the tweets in that location by getting sum and dividing by number of rows
    location_data = data[data["location"] == location]
    total_sentiment_value = sum(list(location_data["target"]))
    return total_sentiment_value/len(location_data)

print(locationSentiment(result[0], "China"))
# tweets_by_location = result[0][result[0]["location"]=="China"]
# print(sum(list(tweets_by_location["target"]))) # total sentiment val
# print(len(tweets_by_location))
# # antartica_tweets
# antartica_tweets.head(10)
# tweets = list(antartica_tweets["text"])
# tweets


1.975084175084175
